# Imports and utils

In [78]:
import pandas as pd
import sys
from dateutil import parser
from io import StringIO

sys.path.append('../src')
from utils import display_info, display_summary_stats, display_unique_values, display_value_counts, plot_histogram, plot_boxplot


# Explore example for df_sales

In [79]:
data = {'value': [93.14, pd.NA, pd.NA, 94.23, 95.12, pd.NA]}
df = pd.DataFrame(data)
display_info(df)
display_summary_stats(df)
display_unique_values(df, columns=['value'])
display_value_counts(df, columns=['value'])

DataFrame Info:
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   value   3 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes
None


DataFrame Shape:
------------------------------
(6, 1)


Missing Values:
------------------------------
value    3
dtype: int64


Summary Statistics:
------------------------------
        value
count    3.00
unique   3.00
top     93.14
freq     1.00


Unique values in value: [93.14 <NA> 94.23 95.12]


Value counts for value:
value
93.14    1
94.23    1
95.12    1
Name: count, dtype: int64




# Clean holidays

In [134]:
df_holidays = pd.read_csv('../data/raw/holidays_events(india).csv', delimiter=';')

df_holidays.dropna(inplace=True)
df_holidays.drop(columns=['type'], inplace=True)

month_mapping = {
    'jan': 'Jan',
    'feb': 'Feb',
    'mrt': 'Mar',
    'apr': 'Apr',
    'mei': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'aug': 'Aug',
    'sep': 'Sep',
    'okt': 'Oct',
    'nov': 'Nov',
    'dec': 'Dec'
}

df_holidays['date'] = df_holidays['date'].str.replace('/', ' ')
for month_nl, month_en in month_mapping.items():
    df_holidays['date'] = df_holidays['date'].str.replace(month_nl, month_en, regex=True)

start_year = 2011

# Define a function to add the year and format the date
for index, row in df_holidays.iterrows():
    
    if row['date'] == '1 Jan':
        if not ((row['description'] == "New Year's Day") & (start_year == 2017)):
            start_year += 1
    
    # Extract day and month from the date
    day_month = row['date'].split(' ')
    month = day_month[1].lower()
    day = day_month[0]

    # Update the date with the year and reformat
    df_holidays.at[index, 'date'] = pd.to_datetime(f"{start_year}-{month}-{day}", format='%Y-%b-%d')


df_holidays = df_holidays.drop(df_holidays[(df_holidays['description'] == 'Holi') & (df_holidays['transferred'] == 'Gazetted Holiday')].index)
df_holidays.to_csv('../data/processed/holidays_india_clean.csv', index=False)

# Clean Oil

## Split in two files
- monthly (date_year_month, unit, total_price, change, price_monthly
- daily (date, price_daily)

In [145]:
OIL_MONTHLY_CNAME_0 = "date_year_month"
OIL_MONTHLY_CNAME_1 = "unit"
OIL_MONTHLY_CNAME_2 = "total_price"
OIL_MONTHLY_CNAME_3 = "change"

OIL_DAILY_CNAME_0 = "date"
OIL_DAILY_CNAME_1 = "price_daily"

# Read the oil(india) file
with open('../data/raw/oil(india).csv', 'r') as file:
    data = file.readlines()
    data_monthly = [f"{OIL_MONTHLY_CNAME_0};{OIL_MONTHLY_CNAME_1};{OIL_MONTHLY_CNAME_2};{OIL_MONTHLY_CNAME_3}\n"]
    data_daily = [f"{OIL_DAILY_CNAME_0};{OIL_DAILY_CNAME_1}\n"]

    for idx, row in enumerate(data):
        modified_row = row.replace(',', ';')

        if (1 <= idx & idx <= 60):
            data_monthly.append(modified_row)
        if (idx > 60):
            # remove last 2 semicolons
            modified_row = ";".join(modified_row.split(";")[:-2])
            data_daily.append(modified_row + "\n")

df_oil_monthly = pd.read_csv(StringIO(''.join(data_monthly)), sep=';')
df_oil_daily = pd.read_csv(StringIO(''.join(data_daily)), sep=';')

## Clean monthly prices

- Date formats uniform to Y-m
- Added price_monthly (total price / units)

In [146]:
# apr/13 --> apr 2013
df_oil_monthly[OIL_MONTHLY_CNAME_0] = df_oil_monthly[OIL_MONTHLY_CNAME_0].str.replace('/', ' 20')

# apr 2013 --> 2013-04
df_oil_monthly.iloc[:, 0] = df_oil_monthly.iloc[:, 0].apply(lambda x: parser.parse(x).strftime('%Y-%m') if isinstance(x, str) else x)

# add column price per unit
df_oil_monthly['price_monthly'] = round(df_oil_monthly[OIL_MONTHLY_CNAME_2] / df_oil_monthly[OIL_MONTHLY_CNAME_1], 2)


df_oil_monthly.to_csv('../data/processed/oil_india_monthly_clean.csv', index=False)

## Clean daily prices

2 options
- Clean given file (2013/03/16 - 2017/8/31)
- Use external file (2013/01/01 - 2018/12/31)

NaN prices where interpolated. First value was forward filled.

In [147]:
df_oil_daily.interpolate(inplace=True)
df_oil_daily.loc[0, 'price_daily'] = df_oil_daily.loc[1, 'price_daily']  # Missed by interpolate
df_oil_daily.to_csv('../data/processed/oil_india_daily_clean.csv', index=False)

In [171]:
df_oil_daily = pd.read_csv('../data/external/oil-alfredgraph.csv', usecols=['date', 'dcoilwtico'])

df_oil_daily['date'] = pd.to_datetime(df_oil_daily['date'])
df_oil_daily.set_index('date', inplace=True)
idx = pd.date_range(df_oil_daily.index.min(), df_oil_daily.index.max())
df_oil_daily = df_oil_daily.reindex(idx)
df_oil_daily.replace('.', pd.NA, inplace=True)
df_oil_daily['dcoilwtico'] = pd.to_numeric(df_oil_daily['dcoilwtico'], errors='coerce')
df_oil_daily.interpolate(inplace=True)
df_oil_daily.reset_index(inplace=True)
df_oil_daily.columns = ['date', 'price_daily']
df_oil_daily['price_daily'] = df_oil_daily['price_daily'].round(2)
df_oil_daily.loc[0, 'price_daily'] = df_oil_daily.loc[1, 'price_daily']
df_oil_daily.to_csv('../data/processed/oil_india_daily_clean.csv', index=False)

# Clean Sales & Items

In [139]:
df_sales = pd.read_csv('../data/raw/data.csv', low_memory=False, usecols=['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'])
df_sales['onpromotion'].fillna(False, inplace=True)
df_sales.to_csv('../data/processed/sales_clean.csv', index=False)

df_items = pd.read_csv('../data/raw/items.csv', delimiter=';', low_memory=False)
df_items.rename(columns={'Price': 'item_price'}, inplace=True)
df_items.dropna(inplace=True)
df_items.to_csv('../data/processed/items_clean.csv', index=False)

# Clean Stores & Transactions

In [140]:
df_stores = pd.read_csv('../data/raw/stores.csv', delimiter=';', encoding='ISO-8859-1')
df_stores.dropna(subset=['store_nbr'], inplace=True)
df_stores['Store name'] = df_stores['Store name'].str.replace('FC-', '')
df_stores['Store name'] = df_stores['Store name'].str.split('-', expand=True)[0].apply(lambda x: 'Fashion Bazaar' if 'FBB' in x else 'Smart Bazaar')
df_stores.rename(columns={'Store name': 'store_name'}, inplace=True)

# Gather state data
store_states = {
    2: 'Assam',
    3: 'Assam',
    21: 'Odisha',
    22: 'Odisha',
    23: 'Odisha',
    39: 'West Bengal',
    43: 'West Bengal'
}

for store_nbr, state in store_states.items():
    df_stores.loc[df_stores['store_nbr'] == store_nbr, 'state'] = state

df_stores['city'] = df_stores['city'].str.capitalize()
df_stores.to_csv('../data/processed/stores_clean.csv', index=False)

df_trans = pd.read_csv('../data/raw/transactions.csv')
df_trans.to_csv('../data/processed/transactions_clean.csv', index=False)

# One merged version

In [182]:
df_sales = pd.read_csv('../data/processed/sales_clean.csv')
df_items = pd.read_csv('../data/processed/items_clean.csv')
df_stores = pd.read_csv('../data/processed/stores_clean.csv')
df_transactions = pd.read_csv('../data/processed/transactions_clean.csv')
df_holidays = pd.read_csv('../data/processed/holidays_india_clean.csv')
df_oil_daily = pd.read_csv('../data/processed/oil_india_daily_clean.csv')
df_oil_monthly = pd.read_csv('../data/processed/oil_india_monthly_clean.csv')


# Merge sales & items
df = pd.merge(df_sales, df_items, on='item_nbr')

# Merge transactions
df = pd.merge(df, df_transactions, on=['date', 'store_nbr'])

# Merge stores
df = pd.merge(df, df_stores, on='store_nbr')

# Add total sales
df['total_sales'] = df['unit_sales'] * df['item_price']

# Add weekday
df['date'] = pd.to_datetime(df['date']).dt.date
df['weekday'] = df['date'].apply(lambda x: x.weekday())

# Merge holidays
df_holidays['date'] = pd.to_datetime(df_holidays['date']).dt.date
df = pd.merge(df, df_holidays, on='date', how='left')

# Merge oil_daily
df_oil_daily['date'] = pd.to_datetime(df_oil_daily['date']).dt.date
df = pd.merge(df, df_oil_daily, on='date', how='left')
df.rename(columns={'price_daily': 'oil_price_daily'}, inplace=True)

# Merge oil_monthly
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df_oil_monthly['date_year_month'] = pd.to_datetime(df_oil_monthly['date_year_month'])
df_oil_monthly['month'] = df_oil_monthly['date_year_month'].dt.month
df_oil_monthly['year'] = df_oil_monthly['date_year_month'].dt.year
df = pd.merge(df, df_oil_monthly, on=['month', 'year'])
df.drop(['month', 'year'], axis=1, inplace=True)
df.rename(columns={'price_monthly': 'oil_price_monthly', 'unit': 'oil_units', 'total_price': 'oil_total_price', 'change': 'oil_change'}, inplace=True)


df.to_csv('../data/processed/merged_clean.csv', index=False)

In [ ]:
# One split in train & test